<a href="https://colab.research.google.com/github/ampazio/hallow-universe/blob/main/Code_nodes_edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pierwszy krok to analiza leksykalna w MAXQDA**.

Najpierw otwieramy MAXDiction > Słownik. Tam dodajemy kategorie. Jedna kategoria to jeden kod (pamiętamy, żeby nie było spacji w kilkuczłonowych nazwach). Do każdej kategorii dodajemy pozycje, jakie słownik powinien wyszukiwać. Np. KATEGORIA: JonathanRoumie > pozycja: Jonathan Roumie.

Potem przeprowadzamy analizę treści opartą na słowniku i kodujemy dokumenty wynikami.

**Najpierw potrzebujemy dokonać ekstrakcji naszych kodów z Maxqda**. Pilnujemy, żeby każdy kod stanowił JEDNO słowo (brak spacji).

Aktywujemy wszystkie dokumenty i wszystkie kody.

Następnie idziemy do zakładki Analiza > Kompleksowe wyszukiwanie segmentów.
W oknie, które się otwiera, wybieramy opcję "Intersekcja (zestaw)", zaznaczamy wszystkie kody i ustawiamy minimalną liczbę kodów na 2.

Następnie w zakładce Wyszukane segmenty w dolnej części panelu klikamy "Eksportuj jako plik Excel". Wybieramy opcję "inne kody przypisane do segmentu". Zapisujemy plik na dysku.

Otwieramy plik w Excelu i zapisujemy go jako .csv UTF-8

Ten csv wgrywamy do COLABa i wstawiamy ścieżkę do niego do poniższego skryptu.

In [ ]:
## PRÓBA CZYSZCZENIA PLIKU WYEKSPORTOWANEGO Z MAXQDY

import pandas as pd

# Wczytaj plik CSV
file_path = '/content/MAXQDA 2_KODY.csv'  # Zmień na ścieżkę do Twojego pliku
data = pd.read_csv(file_path, sep=';', encoding='utf-8')

# Zmień nazwę kolumny B
data.rename(columns={'Inne kody przypisane do segmentu': 'kod'}, inplace=True)

# Oczyść dane w kolumnie 'kod'
if 'kod' in data.columns:
    data['kod'] = data['kod'].astype(str).str.replace(r'\(Waga: 0\)', '', regex=True)  # Usuń frazy "(Waga: 0)"
    data['kod'] = data['kod'].str.replace('"', '')  # Usuń zbędne cudzysłowy
    data['kod'] = data['kod'].str.split().apply(lambda x: ';'.join(filter(None, x)))  # Połącz pozostałe słowa średnikami

# Zapisz przetworzony plik
output_path = 'przetworzony_plik.csv'  # Ścieżka do pliku wyjściowego
data.to_csv(output_path, sep=';', index=False, encoding='utf-8')

print(f"Plik został przetworzony i zapisany jako {output_path}")



Plik został przetworzony i zapisany jako przetworzony_plik.csv


Mamy już wyczyszczony plik, który możemy wstawić do poniższego skryptu.

In [ ]:
# POPRAWNY Kod mapujący relacje między kodami dla każdego segmentu

import pandas as pd

# Załaduj dane, używając średnika jako separatora
data = pd.read_csv('/content/przetworzony_plik.csv', delimiter=';')

# Sprawdź nazwy kolumn
print(data.columns)

# Usuń nadmiarowe spacje w nazwach kolumn
data.columns = data.columns.str.strip()

# Sprawdź, czy 'kod' nie zawiera pustych wartości
data['kod'] = data['kod'].fillna('')  # Zastępuje puste wartości pustymi stringami

# Rozdziel kolumnę "kod" na osobne kody (teraz będą to listy ID)
data['kod'] = data['kod'].str.split(';')

# Stwórz listę wszystkich unikalnych kodów (aktorów z kolumny 'kod')
all_codes = set()
for codes in data['kod']:
    all_codes.update(codes)

# Przypisz unikalne numery ID do każdego kodu
code_to_id = {code: idx + 1 for idx, code in enumerate(all_codes)}

# Zamień kody na ich ID w kolumnie 'kod'
data['kod'] = data['kod'].apply(lambda x: [code_to_id[code] for code in x])

# Tworzymy listę krawędzi (relacje między kodami w ramach tego samego segmentu)
edges = []

# Dla każdego wiersza w danych
for idx, row in data.iterrows():
    codes = row['kod']  # Lista kodów przypisanych do segmentu

    # Tworzymy wszystkie unikalne pary kodów w ramach jednego segmentu
    for i in range(len(codes)):
        for j in range(i + 1, len(codes)):
            edges.append([codes[i], codes[j], 'Undirected', 1])

# Tworzymy DataFrame dla krawędzi
edges_df = pd.DataFrame(edges, columns=['Source', 'Target', 'Type', 'Weight'])

# Tworzymy DataFrame dla węzłów (code_id i code_name)
nodes = pd.DataFrame({
    'Id': [code_to_id[code] for code in all_codes],
    'Label': [code for code in all_codes]
})

# Zapisz pliki CSV
nodes.to_csv('nodes.csv', index=False, sep=';')  # Węzły (kody)
edges_df.to_csv('edges.csv', index=False, sep=';')  # Krawędzie (relacje między kodami)

print("Pliki nodes.csv i edges.csv zostały wygenerowane.")


Index(['Segment', 'kod'], dtype='object')
Pliki nodes.csv i edges.csv zostały wygenerowane.


Wrzucamy oba pliki do Gephi.
Zmieniamy układ m.in. przy użyciu algorytmu Fruchterman Reingold. Ustawiamy wielkość nodes, tak żeby były zależne od stopnia usieciowienia. Wyodrębniamy communities przy użyciu statystyki Modularity itp.